# Learning OpenAI-Gym and Stable-Baseline

Liu Shuojiang

## Import Dependencies

In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Load Environment

In [2]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [3]:
env.reset()

array([-0.01809891,  0.00805074,  0.00980379,  0.03617031], dtype=float32)

In [4]:
env.action_space.sample()

1

In [5]:
env.observation_space.sample()

array([ 1.6654559e+00,  3.0435396e+38, -7.2203353e-02, -1.8139597e+38],
      dtype=float32)

In [6]:
env.step(1)

(array([-0.0179379 ,  0.20303074,  0.0105272 , -0.25340337], dtype=float32),
 1.0,
 False,
 {})

In [7]:
episodes = 5
for episode in range(1, episodes + 1):
    # env.reset() reset the environment and obtain initial observations
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()  # visualize the environment
        action = env.action_space.sample()
        # env.step() apply an action to the environment
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode:{episode}, Score:{score}')
env.close()  # close down the render frame

Episode:1, Score:12.0
Episode:2, Score:14.0
Episode:3, Score:23.0
Episode:4, Score:12.0
Episode:5, Score:60.0


## Understanding the Environment

[https://www.gymlibrary.dev/environments/classic_control/cart_pole/](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)

In [8]:
env.action_space

Discrete(2)

In [9]:
env.action_space.sample()

1

In [10]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [11]:
env.observation_space.sample()

array([ 5.2408487e-01, -1.6591366e+38, -3.7983936e-01,  1.4704907e+38],
      dtype=float32)

## Train an RL model

In [12]:
log_path = os.path.join('Training', 'Logs')
log_path

'Training\\Logs'

In [13]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda : env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [16]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 57   |
|    iterations      | 1    |
|    time_elapsed    | 35   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 85          |
|    iterations           | 2           |
|    time_elapsed         | 47          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009840605 |
|    clip_fraction        | 0.121       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00394    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.55        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0205     |
|    value_loss           | 52.6        |
-----------------------------------------
---

## Save and Reload Model

In [14]:
# Don't create new folder manually!!!
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO-Model-CartPole')

In [18]:
model.save(PPO_Path)

In [19]:
del model

In [15]:
model = PPO.load(PPO_Path, env=env)

In [22]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 316  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------


## Evaluation

In [23]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

D:\Anaconda\envs\RL\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [24]:
env.close()

## Test Model

In [25]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)  # using the model
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode:{episode}, Score:{score}')
env.close()

Episode:1, Score:[200.]
Episode:2, Score:[200.]
Episode:3, Score:[200.]
Episode:4, Score:[200.]
Episode:5, Score:[200.]


In [26]:
model.predict(obs)

(array([0], dtype=int64), None)

## View Logs in Tensorboard

In [16]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [17]:
!tensorboard --logdir={training_log_path}
# localhost:6006

^C


## Adding a callback to the training stage

In [18]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [19]:
save_path = os.path.join('Training', 'Saved models')

In [20]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [21]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [22]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 132  |
|    iterations      | 1    |
|    time_elapsed    | 15   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 148         |
|    iterations           | 2           |
|    time_elapsed         | 27          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008622104 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000683   |
|    learning_rate        | 0.0003      |
|    loss                 | 6.04        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0169     |
|    value_loss           | 45.1        |
-----------------------------------------
---

D:\Anaconda\envs\RL\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.006910893 |
|    clip_fraction        | 0.0576      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.61       |
|    explained_variance   | 0.266       |
|    learning_rate        | 0.0003      |
|    loss                 | 37.7        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.016      |
|    value_loss           | 68.5        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


## Changing Policies

In [23]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [24]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cuda device


In [25]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 267  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 175         |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013966923 |
|    clip_fraction        | 0.201       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.000305   |
|    learning_rate        | 0.0003      |
|    loss                 | 3.53        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.022      |
|    value_loss           | 18.5        |
-----------------------------------------
---

D:\Anaconda\envs\RL\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=199.40 +/- 1.20
Episode length: 199.40 +/- 1.20
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 199         |
|    mean_reward          | 199         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.007557491 |
|    clip_fraction        | 0.1         |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.566      |
|    explained_variance   | 0.523       |
|    learning_rate        | 0.0003      |
|    loss                 | 15.2        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0161     |
|    value_loss           | 43.7        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 136   |
|    iterations      | 5     |
|    time_elapsed    | 74    |


## Using another Algorithm

In [26]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)